# **1. Llamada a la API de OpenAI**

Para cada búsqueda realizada de los distintos empleos en Madrid hemos obtenido varias ofertas de trabajo, de las cuales tenemos su correspondiente descripción. Para cada descripción se pretende obtener cúanto se ajusta dicha oferta al puesto para el cual se realizó la búsqueda, por ejemplo cuanto se ajusta la descripción a un puesto de Data Science, cuál es el nivel de inglés si se pidiera, los años de experiencia si se pidieran, las hard skill necesarias y las hard skills deseables.

Para llevar a cabo esta tarea usaremos IA generativa a través de la API de OpenAI, la cual se encargará de llevar a cabo la labor de extraer de cada oferta la información deseada.

In [1]:
%load_ext autoreload
%autoreload 2

#from googletrans import Translator
from difflib import SequenceMatcher
from dotenv import load_dotenv
import ast
import pandas as pd
import pickle
import json
import os

import sys
sys.path.append("../")

from src import funciones_transformacion as f_transform
from src import funciones_bbdd as f_bbdd

load_dotenv()

True

La función de `extracion_datos_descripciones` para cada uno de los archivos finales obtenidos en la parte de extracción se encargá de extraer de la descripción los años de experiencia que piden, el nivel de inglés, las hard skills necesarias, las hard skill deseables y un porcentaje de ajuste. Este porcetaje representa cuanto se ajusta la descripción de la oferta en cuanto al puesto para el que se realizó la busqueda, por ejemplo, Data analyst. Todos estos datos se añadirán a un json que se irá almacenando en la carpeta `data/response_openai` en este mismo directorio.

In [34]:
f_transform.extracion_datos_descripciones()

True
Analizando el archivo -infojobs_data_analyst.csv- de la categoria: Data analyst, con shape: (111, 9)
Analizando el archivo -infojobs_data_engineer.csv- de la categoria: Data engineer, con shape: (83, 9)
Analizando el archivo -infojobs_data_science.csv- de la categoria: Data science, con shape: (61, 9)
Analizando el archivo -linkedin_data_analyst.csv- de la categoria: Data analyst, con shape: (905, 8)
Analizando el archivo -linkedin_data_engineer.csv- de la categoria: Data engineer, con shape: (854, 8)
Analizando el archivo -linkedin_data_science.csv- de la categoria: Data science, con shape: (763, 8)


_________________________________________________

## **2. Limpieza de los JSON**

Una vez tenemos las skills de cada oferta hay que hacer un primer filtrado general de estas skills pues en total tenemos alrededor de 3000. Por lo tanto vamos a agrupar skills que se refieren a una herramienta pero han sido recogidas con distintos nombres en cada oferta como: PowerBi, Power BI, experto en PowerBi, todo ello lo pasaremos a un mismo término y asi se pretende reducir el número de skill.

In [57]:
f_transform.extraer_y_guardar_skills_filtradas()

Filtrando las skills del archivo: transformed_infojobs_data_analyst.json
Filtrando las skills del archivo: transformed_infojobs_data_engineer.json
Filtrando las skills del archivo: transformed_infojobs_data_science.json
Filtrando las skills del archivo: transformed_linkedin_data_analyst.json
Filtrando las skills del archivo: transformed_linkedin_data_engineer.json
Filtrando las skills del archivo: transformed_linkedin_data_science.json

Skills filtradas y guardadas correctamente.


Comprobamos cuantas skills en total había antes y después del filtrado:

In [58]:
lista_skills = f_transform.contador_skills("data/response_openai/")

lista_skills = f_transform.contador_skills("data/skills_filtradas/")


Contando skills de los archivos: ['transformed_infojobs_data_analyst.json', 'transformed_infojobs_data_engineer.json', 'transformed_infojobs_data_science.json', 'transformed_linkedin_data_analyst.json', 'transformed_linkedin_data_engineer.json', 'transformed_linkedin_data_science.json']
Se han encontrado 2951 skills diferentes.

Contando skills de los archivos: ['filtered_transformed_infojobs_data_analyst.json', 'filtered_transformed_infojobs_data_engineer.json', 'filtered_transformed_infojobs_data_science.json', 'filtered_transformed_linkedin_data_analyst.json', 'filtered_transformed_linkedin_data_engineer.json', 'filtered_transformed_linkedin_data_science.json']
Se han encontrado 2282 skills diferentes.



Hemos conseguido reducir el número total de skills unas 700, sin embargo, seguir manteniendo 2300 skills es inviable, pues para que el recomendador funcione el usuario deberá chequear si tiene conocimientos en cada una de ellas o no, y revisar una lista de más de 2000 skills no es una opción realista.

Por lo tanto, para cada uno de los tres empleos vamos a ver cuales son las skills que más se repiten y haremos una lista con únicamente las más importante para cada oferta de trabajo. Tendremos que filtrar nuevamente las ofertas y dejar solo las skills que estén en la lista de hard skills generada en el paso anterior y la cual podremos encontrar en `data/lista_hard_skills.pkl`.

In [61]:
skills_relevantes = f_transform.filtrar_skills_min_apariciones(5)

Se han guardado 112 skills relevantes en 'data/lista_hard_skills.pkl'
Se han categorizado las skills y almacenado en 'data/diccionario_skills_categorizadas.pkl'


In [65]:
skills_relevantes = f_transform.filtrar_skills_min_apariciones(5)
f_transform.filtracion_hard_skills_relevantes()

Se han guardado 112 skills relevantes en 'data/lista_hard_skills.pkl'
Se han categorizado las skills y almacenado en 'data/diccionario_skills_categorizadas.pkl'

Limpiando las skills de las ofertas, manteniendo solo las más relevantes...



100%|██████████| 200/200 [00:03<00:00, 63.89it/s] 


Completado!


Comprobamos que efectivamente tras el filtrado nos ha quedado el mismo número de skills que en la lista:

In [67]:
lista_skills = f_transform.contador_skills("data/skills_filtradas_relevantes/")

Contando skills de los archivos: ['relevant_filtered_transformed_infojobs_data_analyst.json', 'relevant_filtered_transformed_infojobs_data_engineer.json', 'relevant_filtered_transformed_infojobs_data_science.json', 'relevant_filtered_transformed_linkedin_data_analyst.json', 'relevant_filtered_transformed_linkedin_data_engineer.json', 'relevant_filtered_transformed_linkedin_data_science.json']
Se han encontrado 112 skills diferentes.



____

# **3. Carga de datos en MongoDB**

Una vez hemos limpiado las skills de todas las ofertas de trabajo y las hemos almacenado en los correspondientes `json` el siguiente paso es subir todos estos datos a una base de datos en la nube. En este caso al ser tablas no relacionales hemos escogido MongoDB y los datos se subiran a MongoDB Atlas.

In [68]:
f_bbdd.cargar_datos_mongo()
f_bbdd.cargar_skills_mongo()

Pinged your deployment. You successfully connected to MongoDB!
Insertando file: relevant_filtered_transformed_infojobs_data_analyst.json
Insertando file: relevant_filtered_transformed_infojobs_data_engineer.json
Insertando file: relevant_filtered_transformed_infojobs_data_science.json
Insertando file: relevant_filtered_transformed_linkedin_data_analyst.json
Insertando file: relevant_filtered_transformed_linkedin_data_engineer.json
Insertando file: relevant_filtered_transformed_linkedin_data_science.json
---- Insercion en la base de datos finalizada ----

Pinged your deployment. You successfully connected to MongoDB!
Skills categorizadas insertadas en la base de datos correctamente.
